In [225]:
#%pip install xgboost
#%pip install scikit-learn

In [226]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [227]:
df = pd.read_csv("Resume.csv")
display(df.head(5),df.tail(5))

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


,ID,Resume_str,Resume_html,Category
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2483,37473139,STOREKEEPER II Professional Sum...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


### Check for duplicates

In [229]:
df.duplicated().sum()

np.int64(0)

### Drop columns 'ID' and 'Resume_html'

In [230]:
df.drop(['ID','Resume_html'], axis=1, inplace=True)

### Cleaned Data

In [231]:
df

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR
...,...,...
2479,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,AVIATION
2480,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",AVIATION
2481,GEEK SQUAD AGENT Professional...,AVIATION
2482,PROGRAM DIRECTOR / OFFICE MANAGER ...,AVIATION


### Create a new column 'label' that indicates whether a candidate passed the interview (1: passed; 0: fail)
* Assume that only 10% of the total applications passed: 248 out of 2483.
* So there's 248 '1s' and the rests is '0s'

In [232]:
# Calculate the number of rows and the target number of 1s
num_rows = len(df)
num_ones = int(0.3 * num_rows)

# Create a NumPy array with the desired distribution of 0s and 1s
new_column_values = np.zeros(num_rows)
new_column_values[:num_ones] = 1
np.random.shuffle(new_column_values)

# Add the new column to the DataFrame
df['Label'] = new_column_values

### Our revised dataset
* There are 3 columns: 
    * Resume_str: contains the resume text only in string format
    * Category: type of industry of the candidates
    * Lable: passed or failed the interview

In [233]:
df

,Resume_str,Category,Label
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,0.0
1,"HR SPECIALIST, US HR OPERATIONS ...",HR,0.0
2,HR DIRECTOR Summary Over 2...,HR,1.0
3,HR SPECIALIST Summary Dedica...,HR,0.0
4,HR MANAGER Skill Highlights ...,HR,1.0
...,...,...,...
2479,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,AVIATION,1.0
2480,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",AVIATION,0.0
2481,GEEK SQUAD AGENT Professional...,AVIATION,0.0
2482,PROGRAM DIRECTOR / OFFICE MANAGER ...,AVIATION,0.0


### Column 'Resume_str'

In [234]:
df['Resume_str'][0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

Observation: very straightforward. This columns contain the resume in text in string format

### Column 'Category'

In [235]:
display(df['Category'].unique(), df['Category'].nunique())

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

24

Observations: there are 24 unique industry category 

In [236]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

Observation: As we can see from the number of times each industry category appeared, there are 3 categories that appeared significantly less than the others. These 3 categories are: 'Agriculture', 'Automobile', and 'BPO'

### Column 'Label'

In [237]:
df['Label'].value_counts()

Label
0.0    1739
1.0     745
Name: count, dtype: int64

Observation: 10% of the applicants (248) has passed the interview indicated by the label '1s'

## Cleaning data

In our dataset, the column 'Resume_str' is the predictor variable. The 'Category' and 'Label' columns are the outcome variable.
In this data cleaning part, we want to focus on the predictor variable, which is the 'Resume_str' column. 

As the text in a resume may consists of many types of words/letters, would want to remove a few things that are irrelevant, such as:
1. URLS
2. Hashtags
3. Mentions
4. Punctuation and Special Letters
5. Non-ASCII characters 
6. Spaces
7. Stopwords (will be removed in the tf-idf section later)

In [238]:
import re

def cleanResume(txt):
    #URls: start with http, then matches one or more non-white space characters
    cleanTxt = re.sub('http\\S+',' ', txt)  
    #Hashtags: start with #, then matches one or more non-white space characters
    cleanTxt = re.sub('#\\S+',' ', cleanTxt)
    #Mentions: start with #, then matches one or more non-white space characters
    cleanTxt = re.sub('@\\S+',' ', cleanTxt)
    #Remove punctuation marks and other special characters from the text
    cleanTxt = re.sub(r'[^\w\s]', ' ', cleanTxt)
    #Remove any non-ASCII characters 
    cleanTxt = re.sub(r'[^\x00-\x7f]',' ', cleanTxt)    
    #Substitute 1 or more than 1 white space with 1 space
    cleanTxt = re.sub('\\s+',' ', cleanTxt)
    return cleanTxt

In [239]:
df['Resume_str'] = df['Resume_str'].apply(lambda x: cleanResume(x))
df['Resume_str']

0        HR ADMINISTRATOR MARKETING ASSOCIATE HR ADMIN...
1        HR SPECIALIST US HR OPERATIONS Summary Versat...
2        HR DIRECTOR Summary Over 20 years experience ...
3        HR SPECIALIST Summary Dedicated Driven and Dy...
4        HR MANAGER Skill Highlights HR SKILLS HR Depa...
                              ...                        
2479     RANK SGT E 5 NON COMMISSIONED OFFICER IN CHAR...
2480     GOVERNMENT RELATIONS COMMUNICATIONS AND ORGAN...
2481     GEEK SQUAD AGENT Professional Profile IT supp...
2482     PROGRAM DIRECTOR OFFICE MANAGER Summary Highl...
2483     STOREKEEPER II Professional Summary The purpo...
Name: Resume_str, Length: 2484, dtype: object

### TF-IDF the column 'Resume_str'

Essentially, tf-idf assess the importance of words in a document relative to a collection of documents. It helps to identify the most relevant words in a text by considering both how often a word appears in a document (term frequency) and how rare it is across the entire collection of documents (inverse document frequency). 

In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df['Resume_str'])

finaltext = tfidf.transform(df['Resume_str'])

In [241]:
finaltext

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 805156 stored elements and shape (2484, 40033)>

Observations: Our predictor variable has been transformed into a matrix format where there are 40033 columns (each column represents 1 word) and the values are the tf-idf value. 

## Data Processing
There are 2 variables we would want to predict:
* 'Category': to predict which category a resume belongs to (classification)
* 'Label': to predict the probability that a candidate is likely to do well in the interview (regression)

## Modeling & Evaluation (Predicting Category)

In [242]:
# Import Library (Classification)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [243]:
# Preparing Data Training and Testing
x = finaltext
y = df['Category']
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=2024)

In [244]:
# Model Selection (Classification)
logreg = LogisticRegression()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

### Model Benchmarking : K-Fold

In [245]:
models = [logreg,knn,dt,rf]
score=[]
rata=[]
std=[]

for i in models:
    skfold=StratifiedKFold(n_splits=5)
    model_cv=cross_val_score(i,x_train,y_train,cv=skfold,scoring='accuracy')
    score.append(model_cv)
    rata.append(model_cv.mean())
    std.append(model_cv.std())
    
pd.DataFrame({'model':['Logistic Regression', 'KNN', 'Decision Tree', 'Random Forest'],'mean accuracy':rata,'sdev':std}).set_index('model').sort_values(by='mean accuracy',ascending=False)

,mean accuracy,sdev
model,,
Logistic Regression,0.645702,0.014395
Random Forest,0.624043,0.026637
Decision Tree,0.587854,0.037652
KNN,0.528432,0.016438


In [246]:
from sklearn.metrics import accuracy_score

clf = LogisticRegression()
clf.fit(x_train,y_train)
ypred = clf.predict(x_test)
print(accuracy_score(ypred,y_test))

0.6317907444668008


## Modeling & Evaluation (Predicting Label)

In [247]:
# Import Library (Regression)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [248]:
# Preparing Data Training and Testing
x = finaltext
y = df['Label']
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=2024)

Since the output is a probability (values between 0 and 1), we'll use logistic regression as our prediction model

### Prediction model

In [249]:
rgr = LogisticRegression()
rgr.fit(x_train,y_train)
# Make predictions and get probabilities for the positive class
ypred = rgr.predict_proba(x_test)[:,1] 

# Apply threshold using vectorized operation (more efficient)
threshold = 0.5
ypred_test = (ypred >= threshold).astype(int)

#Check accuracy
print(accuracy_score(ypred_test,y_test))
ypred

0.7002012072434608


array([0.27819281, 0.26806649, 0.24893508, 0.31624762, 0.27345449,
       0.299385  , 0.28955587, 0.27784615, 0.22366716, 0.30468491,
       0.36693663, 0.22355217, 0.33188411, 0.28174472, 0.33858343,
       0.36437705, 0.25263357, 0.20175478, 0.39678615, 0.34440803,
       0.33554614, 0.31658072, 0.28074168, 0.27029835, 0.29530373,
       0.41179247, 0.35770775, 0.33150374, 0.34012861, 0.22492473,
       0.33151017, 0.21933   , 0.32957241, 0.41221412, 0.39377672,
       0.3692756 , 0.36515801, 0.37937422, 0.24400998, 0.27710045,
       0.23954583, 0.40330415, 0.29954823, 0.19563338, 0.26393336,
       0.25933848, 0.29264151, 0.30406627, 0.29482451, 0.39151572,
       0.19308105, 0.31483052, 0.29566965, 0.32674996, 0.26175953,
       0.19371041, 0.24645104, 0.25674508, 0.27879892, 0.29157736,
       0.38104484, 0.32427129, 0.37220395, 0.27513751, 0.25899524,
       0.23477284, 0.31769482, 0.25684068, 0.23731928, 0.2954436 ,
       0.36672481, 0.3254136 , 0.40689777, 0.31593728, 0.26123

## App preparation

In [254]:
# Saving our tf-idf funtion and our clf trained models for later use
import pickle
pickle.dump(tfidf, open('tfidf.pkl','wb'))
pickle.dump(clf, open('clf.pkl','wb'))
pickle.dump(rgr, open('rgr.pkl','wb'))

In [259]:
myresume = df['Resume_str'][700]

In [260]:
import pickle

# Load the trained classifier
clf = pickle.load(open('clf.pkl','rb'))
rgr = pickle.load(open('rgr.pkl','rb'))
tfidf = pickle.load(open('tfidf.pkl','rb'))

# Clean the input resume
input_resume = cleanResume(myresume)

# Transform the input resume using trained Tf-idf
input_finaltext = tfidf.transform([input_resume])

# Make predictions using the loaded classifier
predict_category = clf.predict(input_finaltext)[0]
predict_label = rgr.predict_proba(input_finaltext)[:,1][0]

print('Predicted Category:', predict_category)
print('Predicted Label:', predict_label)

Predicted Category: HEALTHCARE
Predicted Label: 0.34231873761778214
